In [4]:
import geopandas as gpd
gpd.use_pygeos=False

/home/lucas/miniconda3/envs/sips/lib/python3.7/site-packages/geopandas/_compat.py:78: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
import os, sys
from matplotlib import colors
import matplotlib.cm as cm

In [8]:
root = os.path.abspath(os.path.join(os.getcwd(),'..'))

In [30]:
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [41]:
gdf = gpd.read_file(os.path.join(root,'data','ABCD_finalized.geojson'))

In [42]:
gdf

,unique_id,area,confidence,install_date,install_date_ints,SPOT_ids_0,S2_ids_0,iso-3166-1,iso-3166-2,land_cover_CORINE_2006,...,land_cover_CDL_2018,SPOT_ids_9,SPOT_ids_10,SPOT_ids_11,SPOT_ids_12,gti,pvout,capacity_mw,match_id,geometry
0,0,13592.361843,B,,,4096:200:1.5:34:8:686_98,,GR,GR-G,12,...,None,None,None,None,None,1995.360685,1602.717004,0.807655,,"POLYGON ((21.61460 38.11466, 21.61466 38.11463..."
1,1,3411.622302,B,,,4096:200:1.5:32:-31:905_2,,DE,DE-RP,12,...,None,None,None,None,None,1241.484724,1050.093750,0.192504,,"POLYGON ((6.37936 50.19216, 6.37939 50.19214, ..."
2,2,20889.852876,D,,,"4096:200:1.5:30:23:938_28,4096:200:1.5:30:23:9...",,GB,GB-NTH,12,...,None,None,None,None,None,1204.594536,1019.778020,1.177707,GBR0002228,"POLYGON ((-0.89647 52.03907, -0.89651 52.03910..."
3,3,4154.511624,B,,,4096:200:1.5:34:9:686_138,,GR,GR-G,20,...,None,None,None,None,None,1991.343052,1598.333974,0.247039,,"POLYGON ((21.65545 38.10883, 21.65545 38.10876..."
4,4,4794.474966,B,,,4096:200:1.5:33:-9:910_15,,CZ,CZ-US,20,...,None,None,None,None,None,1310.517022,1096.845758,0.273404,,"POLYGON ((14.27922 50.51754, 14.27924 50.51744..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68656,68724,1427.270111,B,,,4096:200:1.5:30:-33:739_1,,ES,ES-SA,12,...,None,None,None,None,None,2022.389315,1653.121508,0.083336,EIA58953,"POLYGON ((-5.37615 41.02262, -5.37615 41.02254..."
68657,68725,4154.511624,B,,,4096:200:1.5:34:9:686_138,,GR,GR-G,20,...,None,None,None,None,None,1991.343052,1598.333974,0.247039,EIA62140,"POLYGON ((21.65545 38.10883, 21.65545 38.10876..."
68658,68726,3281.747782,B,,,4096:200:1.5:29:39:695_32,,ES,ES-BA,12,...,None,None,None,None,None,2092.882507,1684.167772,0.194640,,"POLYGON ((-6.21606 38.56812, -6.21606 38.56803..."
68659,68727,13592.361843,B,,,4096:200:1.5:34:8:686_98,,GR,GR-G,12,...,None,None,None,None,None,1995.360685,1602.717004,0.807655,,"POLYGON ((21.61460 38.11466, 21.61466 38.11463..."


In [43]:
gdf['install_date_str'] = gdf['install_date'].str.replace('<2016-06','2000-01-01')
gdf['install_date_str'] = gdf['install_date_str'].str.replace(',','')
gdf['install_date_str'] = gdf['install_date_str'].str[0:10]
gdf['dt_obj'] = pd.to_datetime(gdf['install_date_str'])
gdf['dt_diff'] = (gdf['dt_obj'] - dt.strptime('2016-06-01','%Y-%m-%d')).dt.days

In [44]:
colmap = cm.get_cmap('cool_r',256)
date_delta_max = (dt.strptime('2018-12-31','%Y-%m-%d') - dt.strptime('2016-06-01','%Y-%m-%d')).days

In [45]:
def apply_color(el_diff):
    if np.isnan(el_diff):
        return '#%02x%02x%02x' % (133, 0, 119)
    else:
        rgb = list(
            colmap(
                np.nan_to_num(
                    int((0.2 + 0.8*(max(el_diff/date_delta_max,0)))*256)
                )
            ))
        return '#%02x%02x%02x' % (int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

In [46]:
gdf['hex_color'] = gdf['dt_diff'].apply(apply_color)

In [48]:
gdf = gdf.drop(columns=['S2_ids_0','S2_ids_1','install_date_str','dt_obj','dt_diff','install_date_ints']+[f'SPOT_ids_{ii}' for ii in range(13)])

In [52]:
gdf = gdf.rename(columns={cc:cc.replace('land_cover','lc') for cc in gdf.columns if 'land_cover' in cc})

In [53]:
gdf.columns

Index(['unique_id', 'area', 'confidence', 'install_date', 'iso-3166-1',
       'iso-3166-2', 'lc_CORINE_2006', 'lc_CORINE_2012', 'lc_CORINE_2018',
       'lc_MODIS_2006', 'lc_MODIS_2007', 'lc_MODIS_2008', 'lc_MODIS_2009',
       'lc_MODIS_2010', 'lc_MODIS_2011', 'lc_MODIS_2012', 'lc_MODIS_2013',
       'lc_MODIS_2014', 'lc_MODIS_2015', 'lc_MODIS_2016', 'lc_MODIS_2017',
       'lc_MODIS_2018', 'lc_CDL_2006', 'lc_CDL_2007', 'lc_CDL_2008',
       'lc_CDL_2009', 'lc_CDL_2010', 'lc_CDL_2011', 'lc_CDL_2012',
       'lc_CDL_2013', 'lc_CDL_2014', 'lc_CDL_2015', 'lc_CDL_2016',
       'lc_CDL_2017', 'lc_CDL_2018', 'gti', 'pvout', 'capacity_mw', 'match_id',
       'geometry', 'hex_color'],
      dtype='object')

In [55]:
os.getcwd()

'/home/lucas/repos/solar-pv-global-inventory'

In [56]:
gdf.to_file(os.path.join(os.getcwd(),'data','ABCD_colored.shp'))